In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
from transformers import (
    TFGPT2LMHeadModel,
)
import tensorflow as tf
import keras as K
import numpy as np

/home/rafael.jarczewski/venv-llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-24 15:00:34.079541: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-24 15:00:34.096075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742839234.116129 2808398 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742839234.122238 2808398 cuda_blas.cc:140

In [3]:
model = TFGPT2LMHeadModel.from_pretrained("openai-community/gpt2")

2025-03-24 15:00:39.782970: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-03-24 15:00:39.783016: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:167] env: CUDA_VISIBLE_DEVICES="-1"
2025-03-24 15:00:39.783024: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] CUDA_VISIBLE_DEVICES is set to -1 - this hides all GPUs from CUDA
2025-03-24 15:00:39.783032: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-03-24 15:00:39.783039: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: dl-28
2025-03-24 15:00:39.783043: I external/local_xla/xla/stream_executor/cuda/cuda_di

In [4]:
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  124439808 
 er)                                                             
                                                                 
Total params: 124439808 (474.70 MB)
Trainable params: 124439808 (474.70 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Apply Lora on GPT2

Consultar a classe [TFGPT2MainLayer](https://huggingface.co/transformers/v2.0.0/_modules/transformers/modeling_tf_gpt2.html)

In [6]:
class LoraAttn(tf.keras.layers.Layer):
    """
        Classe que implementa a técnica de LoRA (Low-Rank Adaptation) para camadas de atenção no TensorFlow.

        Esta implementação aplica LoRA na camada de atenção do modelo, adaptando sua função de atenção através da introdução de uma perturbação de baixo-rank, representada pelas matrizes `A` e `B`. Essas matrizes são aprendidas durante o treinamento e controlam a adaptação da camada de atenção original.

        A classe é projetada para ser utilizada diretamente dentro de uma arquitetura de rede neural baseada em Transformer, como o GPT-2, onde o número de heads de atenção e a dimensionalidade das camadas são bem conhecidos.

        A classe realiza as seguintes operações:
        1. Inicializa as matrizes `A` e `B` que formam a adaptação de baixo-rank.
        2. Aplica uma modulação da camada de atenção original usando essas matrizes.
        3. A integração das modificações de LoRA no gráfico computacional do TensorFlow é garantida com a adição de perdas nulas, permitindo que o gradiente seja calculado corretamente durante o treinamento.
    """
    def __init__(self, attn_layer: tf.keras.layers.Layer, rank: int, layer_id:int):
        super(LoraAttn, self).__init__(name="lora_layer")
        self.old_layer = attn_layer
        self.old_weights = self.old_layer.get_weights()
        self.rank = rank

        self.A = self.add_weight(
            shape=(768, self.rank), # 768 refere-se à dimensão da entrada de cada cabeça de atenção (no caso do GPT-2)
            initializer='glorot_uniform',
            trainable=True,
            name=f"lora_qa_{layer_id}"
        )

        self.B = self.add_weight(
            shape=(self.rank, 2304), # 2304 refere-se à dimensão da saída da camada de atenção
            initializer='zeros',
            trainable=True,
            name=f"lora_qb_{layer_id}"
        )

        self.lora_alpha = 1.0
        
    def call(self, inputs):
        """
            Aplica a modulação LoRA na camada de atenção original.
            A camada de atenção original é computada normalmente e depois ajustada com as modificações de baixo-rank introduzidas pelas matrizes `A` e `B`.
            inputs: O tensor de entrada para a camada de atenção.
            Retorna o Tensor modificado que combina a saída da atenção original com a adaptação de baixo-rank.
        """
        # Passo 1: Obtém a saída da camada de atenção original
        original = self.old_layer(inputs)
        # Passo 2: Aplica a modulação de baixo-rank LoRA
        output = tf.matmul(tf.matmul(inputs, self.A), self.B) * self.lora_alpha
        
        # Passo 3: Força a inclusão das novas camadas LoRA no grafo computacional do TensorFlow
        # Isso é necessário para garantir que o TensorFlow reconheça as operações e calcule gradientes corretamente.
        self.add_loss(0.0 * tf.reduce_sum(self.A))
        self.add_loss(0.0 * tf.reduce_sum(self.B))
        
        # Passo 4: Retorna a combinação da saída original com a adaptação LoRA
        return original + output

In [7]:
trainable_params = model.trainable_variables
total_trainable_params = sum([tf.size(variable).numpy() for variable in trainable_params])

print(f"Total de parâmetros treináveis: {total_trainable_params}")

Total de parâmetros treináveis: 124439808


In [8]:
def apply_lora():
    for i, tfblock in enumerate(model.transformer.h):
        original_attn = tfblock.attn.c_attn  # Acessa a camada original de atenção
        original_attn.trainable = False  # Congela a camada original de atenção
        
        # Congela as variáveis da camada original e remove o gradiente
        for var in original_attn.variables:
            var.assign(tf.stop_gradient(var))  # Remove o gradiente
            var._trainable = False  # Congela os pesos da camada original

        # Cria a camada LoRA e a integra à camada original de atenção
        lora_layer = LoraAttn(original_attn, rank=8, layer_id=i)
        for var in lora_layer.variables:
            # Apenas os pesos LoRA são treináveis
            if "lora" in var.name:
                var._trainable = True
            else:
                var._trainable = False  # Congela qualquer outro peso herdado

        # Substitui a camada original pela camada LoRA adaptada
        setattr(tfblock.attn, "c_attn", lora_layer)


    # Congela os embeddings e a última camada LayerNorm
    model.transformer.wte.trainable = False  # Embeddings de tokens
    model.transformer.wpe.trainable = False  # Embeddings de posições
    model.transformer.ln_f.trainable = False  # Última LayerNorm

    # Congela as camadas de normalização e feedforward do modelo
    for i, tfblock in enumerate(model.transformer.h):
        tfblock.ln_1.trainable = False  # LayerNorm 1
        tfblock.ln_2.trainable = False  # LayerNorm 2
        tfblock.attn.c_proj.trainable = False  # Projeção da atenção
        tfblock.mlp.c_fc.trainable = False  # Feedforward do MLP
        tfblock.mlp.c_proj.trainable = False  # Projeção final do MLP

    # Recompila o modelo para registrar as camadas LoRA no gráfico computacional
    optimizer = model.optimizer if model.optimizer else tf.keras.optimizers.Adam()
    model.compile(optimizer=optimizer)
    return model

In [9]:
model = apply_lora()

In [10]:
for i, tfblock in enumerate(model.transformer.h):
    print(f"Bloco {i} - {tfblock.attn.c_attn.name}, Treinável: {tfblock.attn.c_attn.trainable}")


Bloco 0 - lora_layer, Treinável: True
Bloco 1 - lora_layer, Treinável: True
Bloco 2 - lora_layer, Treinável: True
Bloco 3 - lora_layer, Treinável: True
Bloco 4 - lora_layer, Treinável: True
Bloco 5 - lora_layer, Treinável: True
Bloco 6 - lora_layer, Treinável: True
Bloco 7 - lora_layer, Treinável: True
Bloco 8 - lora_layer, Treinável: True
Bloco 9 - lora_layer, Treinável: True
Bloco 10 - lora_layer, Treinável: True
Bloco 11 - lora_layer, Treinável: True


In [11]:
print("Lista de Variáveis Treináveis:")
for var in model.trainable_variables:
    print(f"{var.name}, Shape: {var.shape}")


Lista de Variáveis Treináveis:
lora_qa_0:0, Shape: (768, 8)
lora_qb_0:0, Shape: (8, 2304)
lora_qa_1:0, Shape: (768, 8)
lora_qb_1:0, Shape: (8, 2304)
lora_qa_2:0, Shape: (768, 8)
lora_qb_2:0, Shape: (8, 2304)
lora_qa_3:0, Shape: (768, 8)
lora_qb_3:0, Shape: (8, 2304)
lora_qa_4:0, Shape: (768, 8)
lora_qb_4:0, Shape: (8, 2304)
lora_qa_5:0, Shape: (768, 8)
lora_qb_5:0, Shape: (8, 2304)
lora_qa_6:0, Shape: (768, 8)
lora_qb_6:0, Shape: (8, 2304)
lora_qa_7:0, Shape: (768, 8)
lora_qb_7:0, Shape: (8, 2304)
lora_qa_8:0, Shape: (768, 8)
lora_qb_8:0, Shape: (8, 2304)
lora_qa_9:0, Shape: (768, 8)
lora_qb_9:0, Shape: (8, 2304)
lora_qa_10:0, Shape: (768, 8)
lora_qb_10:0, Shape: (8, 2304)
lora_qa_11:0, Shape: (768, 8)
lora_qb_11:0, Shape: (8, 2304)


In [12]:
after_lora_params = model.count_params()
print(f"Total de parâmetros: {after_lora_params}")

Total de parâmetros: 124734720


In [13]:
print(f"Lora parameters add: {after_lora_params - total_trainable_params}")


Lora parameters add: 294912


In [14]:
trainable_params = model.trainable_variables
total_trainable_params = sum([tf.size(variable).numpy() for variable in trainable_params])

print(f"Total de parâmetros treináveis: {total_trainable_params}")

Total de parâmetros treináveis: 294912


Como pode ser notado, apenas as camadas do lora estão habilitadas para o treinamento. Certifique-se sempre que as camadas novas do lora foram inseridas corretamente no grafo computacional e que você congelou todas as outras camadas do modelo.

# Conversão para .tflite

In [ ]:
class GPT2Generator(tf.Module):
    """
        Classe para geração de texto baseada no modelo GPT-2, com suporte à adaptação LoRA para ajustar camadas de atenção do modelo sem a necessidade de re-treinamento completo.

        Esta classe integra o modelo GPT-2 pré-treinado, e oferece suporte para treinamento e inferência. A adaptação LoRA é aplicada nas camadas de atenção, permitindo um ajuste eficiente do modelo para novos dados ou tarefas específicas com um custo computacional reduzido.

        A implementação é otimizada para compatibilidade com LiteRT, permitindo a execução eficiente em dispositivos móveis e incorporados.

        Métodos principais:
        - **apply_lora()**: Aplica a adaptação LoRA nas camadas de atenção do modelo GPT-2, congelando os pesos das camadas originais e permitindo que apenas os pesos das camadas LoRA sejam treinados.
        - **infer()**: Realiza a inferência, ou seja, gera previsões de tokens do modelo com base em entradas fornecidas.
        - **train()**: Realiza o treinamento do modelo, aplicando os gradientes apenas aos pesos LoRA treináveis e utilizando o otimizador AdamW.
        - **get_w()**: Retorna os pesos do modelo, útil para a análise e monitoramento dos parâmetros do modelo durante o treinamento.

        A classe permite personalização da aplicação LoRA nas camadas de atenção e configuração da taxa de aprendizado.
    """
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.optimizer = tf.keras.optimizers.AdamW()
    ## Algumas funções podem ser importantes durante o processo de geração, como nem todas as operações são compativeis durante a conversão, o decorator abaixo ajuda a identificar esse problemas de incompatibilidade e ira mostrar um warning ou compatibility error
    ## Para mais informações consulte https://ai.google.dev/edge/litert/models/authoring
    @tf.lite.experimental.authoring.compatible
    @tf.function(
        input_signature=[
            tf.TensorSpec([None, None], tf.int32),
            tf.TensorSpec([None, None], tf.int32)
        ]
    )
    def infer(self, input_ids, attention_mask):
        """
            Realiza a inferência do modelo para gerar a próxima previsão de tokens.

            Args:
                input_ids (tf.Tensor): IDs de entrada dos tokens.
                attention_mask (tf.Tensor): Máscara de atenção para a sequência de entrada.

            Returns:
                dict: Contém os logits da previsão de tokens.
        """
        # Realiza a previsão, retornando a última previsão de token
        tokens = self.model(input_ids=input_ids, attention_mask=attention_mask, training=False).logits[0:, -1]
        return {
            "logits": tokens
        }

    @tf.lite.experimental.authoring.compatible
    @tf.function(
        input_signature=[
            tf.TensorSpec([None, None], tf.int32),
            tf.TensorSpec([None, None], tf.int32),
            tf.TensorSpec([None, None], tf.int32),
        ]
    )
    def train(self, input_ids, attention_mask, labels):
        """
            Função de treinamento para aplicar gradientes aos pesos LoRA treináveis.
            A função realiza um ciclo de feedforward e backward apenas nos pesos LoRA treináveis.
            Args:
                input_ids (tf.Tensor): IDs dos tokens de entrada.
                attention_mask (tf.Tensor): Máscara de atenção.
                labels (tf.Tensor): Labels para treinamento supervisionado.
            Returns:
                dict: Contém a perda calculada durante o treinamento.
        """
        with tf.GradientTape() as tape:
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                training=True
            )
            loss = outputs.loss
        # Aplica os gradientes aos pesos LoRA treináveis
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

        return {"loss": loss}

    @tf.lite.experimental.authoring.compatible
    @tf.function
    def get_w(self):
        weights = {}
        for var in self.model.weights:
            weights[var.name] = var
        return weights

    @tf.lite.experimental.authoring.compatible
    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def save(self, checkpoint_path):
        tensor_names = [weight.name for weight in self.model.weights if 'lora' in weight.name]
        tensors_to_save = [weight.read_value() for weight in self.model.weights if 'lora' in weight.name]
        tf.raw_ops.Save(
            filename=checkpoint_path,
            tensor_names=tensor_names,
            data=tensors_to_save,
            name='save'
        )
        return {
            "checkpoint_path": checkpoint_path
        }

    @tf.lite.experimental.authoring.compatible
    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def restore(self, checkpoint_path):
        restored_tensors = {}
        for var in self.model.weights:
            if 'lora' in var.name:
                restored = tf.raw_ops.Restore(
                    file_pattern=checkpoint_path,
                    tensor_name=var.name,
                    dt=var.dtype,
                    name='restore'
                )
                var.assign(restored)
                restored_tensors[var.name] = restored
        return {"resposta": "restored"}
    

In [16]:
file_name = "lora_model.tflite"

In [17]:
model2converter = GPT2Generator(model)

Precisamos criar um dicionário com as funções concretas das assinaturas para integrar ao saved model

In [19]:
signatures = {
    "train": model2converter.train.get_concrete_function(),
    "infer": model2converter.infer.get_concrete_function(),
    "get_parameters": model2converter.get_w.get_concrete_function(),
    "save": model2converter.save.get_concrete_function(),
    "restore": model2converter.restore.get_concrete_function(),
}

In [20]:
saved_model_path = "saved_model"

Vamos salvar o nosso modelo através da função saved_model do tensorflow

In [21]:
tf.saved_model.save(
    obj = model2converter,
    export_dir = saved_model_path,
    signatures = signatures,
)

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


Como utilizamos o saved model do tensorflow conseguimos criar um converter através do diretório que armazena as informações do modelo

In [25]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)

Para garantir que as operações que vamos criar terão compatibilidade durante a execução no dispositivos precisamos informar quais conjuntos de operações que nosso modelo utiliza.

In [ ]:
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # Habilita as ops LiteRT.
    tf.lite.OpsSet.SELECT_TF_OPS, # Habilita as ops TensorFlow.
]


In [ ]:
# Habilita resource variables que adiciona garantias de leitura e gravação mais fortes (Opcional)
converter.experimental_enable_resource_variables = True

In [ ]:
tflite_model = converter.convert() # Conversão de fato

In [ ]:
# Salvar o modelo
with open(file_name, "wb") as f:
    f.write(tflite_model)